In [1]:
import pickle
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from qaoa_and_tdvp import QAOAResult
import math


In [20]:
with open('.//results//n4_stats.p', 'rb') as file:
    stats = pickle.load(file)
stats

expectation approximation ratio                      \
                                              mean       std       min   
algorithm        p                                                       
tdvp             1                        0.555697  0.320149  0.133846   
                 2                        0.914538  0.023949  0.873142   
                 3                        0.926812  0.027653  0.895857   
                 4                        0.943432  0.049538  0.869999   
                 5                        0.969683  0.032257  0.935768   
scipy            1                        0.578786  0.215603  0.322760   
                 2                        0.850797  0.141596  0.562920   
                 3                        0.880351  0.073960  0.760516   
                 4                        0.832498  0.142717  0.623810   
                 5                        0.941948  0.074239  0.821507   
gradient_descent 1                             NaN       NaN       NaN   
                 2                             NaN       NaN       NaN   
                 3                             NaN       NaN       NaN   
                 4                             NaN       NaN       NaN   
                 5                             NaN       NaN       NaN   

                             groundspace overlap                      \
                         max                mean       std       min   
algorithm        p                                                     
tdvp             1  0.963225            0.464344  0.373130  0.019871   
                 2  0.947408            0.895209  0.049664  0.831052   
                 3  0.966206            0.866436  0.068250  0.795922   
                 4  0.974706            0.930693  0.062414  0.843536   
                 5  0.999978            0.960623  0.048394  0.906591   
scipy            1  0.795440            0.385312  0.334051  0.015630   
                 2  0.918540            0.786808  0.231791  0.335994   
                 3  0.973016            0.816939  0.087629  0.752143   
                 4  0.981445            0.681178  0.360276  0.043834   
                 5  0.995812            0.910507  0.117591  0.735778   
gradient_descent 1       NaN                 NaN       NaN       NaN   
                 2       NaN                 NaN       NaN       NaN   
                 3       NaN                 NaN       NaN       NaN   
                 4       NaN                 NaN       NaN       NaN   
                 5       NaN                 NaN       NaN       NaN   

                             groundspace sharpness            ...  \
                         max                  mean       std  ...   
algorithm        p                                            ...   
tdvp             1  0.962394              0.331865  0.164583  ...   
                 2  0.943499              0.297600  0.061860  ...   
                 3  0.967577              0.324697  0.090627  ...   
                 4  0.977556              0.232359  0.100608  ...   
                 5  0.999983              0.150136  0.143481  ...   
scipy            1  0.792068              0.346479  0.169024  ...   
                 2  0.946846              0.338920  0.098601  ...   
                 3  0.968308              0.366746  0.101873  ...   
                 4  0.980620              0.308058  0.130118  ...   
                 5  0.996740              0.214588  0.169398  ...   
gradient_descent 1       NaN                   NaN       NaN  ...   
                 2       NaN                   NaN       NaN  ...   
                 3       NaN                   NaN       NaN  ...   
                 4       NaN                   NaN       NaN  ...   
                 5       NaN                   NaN       NaN  ...   

                      gate count               num_fun_calls               \
                             min           max          mean         

In [13]:
stats.columns = ['_'.join(col) for col in stats.columns]

In [14]:
stats

expectation approximation ratio_mean  \
algorithm        p                                         
tdvp             1                              0.555697   
                 2                              0.914538   
                 3                              0.926812   
                 4                              0.943432   
                 5                              0.969683   
scipy            1                              0.578786   
                 2                              0.850797   
                 3                              0.880351   
                 4                              0.832498   
                 5                              0.941948   
gradient_descent 1                                   NaN   
                 2                                   NaN   
                 3                                   NaN   
                 4                                   NaN   
                 5                                   NaN   

                    expectation approximation ratio_std  \
algorithm        p                                        
tdvp             1                             0.320149   
                 2                             0.023949   
                 3                             0.027653   
                 4                             0.049538   
                 5                             0.032257   
scipy            1                             0.215603   
                 2                             0.141596   
                 3                             0.073960   
                 4                             0.142717   
                 5                             0.074239   
gradient_descent 1                                  NaN   
                 2                                  NaN   
                 3                                  NaN   
                 4                                  NaN   
                 5                                  NaN   

                    expectation approximation ratio_min  \
algorithm        p                                        
tdvp             1                             0.133846   
                 2                             0.873142   
                 3                             0.895857   
                 4                             0.869999   
                 5                             0.935768   
scipy            1                             0.322760   
                 2                             0.562920   
                 3                             0.760516   
                 4                             0.623810   
                 5                             0.821507   
gradient_descent 1                                  NaN   
                 2                                  NaN   
                 3                                  NaN   
                 4                                  NaN   
                 5                                  NaN   

                    expectation approximation ratio_max  \
algorithm        p                                        
tdvp             1                             0.963225   
                 2                             0.947408   
                 3                             0.966206   
                 4                             0.974706   
                 5                             0.999978   
scipy            1                             0.795440   
                 2                             0.918540   
                 3                             0.973016   
                 4                             0.981445   
                 5                             0.995812   
gradient_descent 1                                  NaN   
                 2                                  NaN   
                 3                                  NaN   
                 4                                  NaN   
                 5                             

In [4]:
stats_tdvp = stats.xs('tdvp').reset_index(
    col_level=1)  #maybe find a way to include the p values into each measure subtable
stats_tdvp

expectation approximation ratio                                \
   p                            mean       std       min       max   
0  1                        0.555697  0.320149  0.133846  0.963225   
1  2                        0.914538  0.023949  0.873142  0.947408   
2  3                        0.926812  0.027653  0.895857  0.966206   
3  4                        0.943432  0.049538  0.869999  0.974706   
4  5                        0.969683  0.032257  0.935768  0.999978   

  groundspace overlap                               groundspace sharpness  \
                 mean       std       min       max                  mean   
0            0.464344  0.373130  0.019871  0.962394              0.331865   
1            0.895209  0.049664  0.831052  0.943499              0.297600   
2            0.866436  0.068250  0.795922  0.967577              0.324697   
3            0.930693  0.062414  0.843536  0.977556              0.232359   
4            0.960623  0.048394  0.906591  0.999983              0.150136   

   ...    gate count               num_fun_calls                             \
   ...           min           max          mean          std   min     max   
0  ...  1.069340e+06  1.938870e+06     55.000000     7.014271  44.0    62.0   
1  ...  1.101945e+07  1.254455e+08    194.000000   203.399115  56.0   602.0   
2  ...  3.665482e+07  1.293696e+09    548.666667  1003.912081   0.0  2588.0   
3  ...  1.450715e+08  2.956321e+08     83.333333    71.079298   0.0   182.0   
4  ...  5.209588e+08  4.559181e+09    370.000000   621.564478   0.0  1574.0   

        steps                              
         mean          std    min     max  
0   75.666667     7.312090   62.0    82.0  
1  235.000000   232.458168   79.0   701.0  
2  774.600000  1247.291987   85.0  3000.0  
3  157.000000    45.731098  126.0   225.0  
4  850.000000   826.339519  179.0  1773.0  

[5 rows x 33 columns]

In [27]:
px.scatter(
    data_frame=stats['expectation approximation ratio'].reset_index(),
    x='p',
    y='mean',
    error_y='std',
    color='algorithm',
)

In [10]:
algs = {'tdvp','scipy'}
plots = list()
for alg in algs:
    df = stats.xs(alg)['expectation approximation ratio'].reindex()
    plots.append(go.Scatter(
        x=list(range(1, 6)),
        y=df['mean'],
        name=alg,
        # error_y='std',
    )
    )
plot = go.Figure(
    data = plots
)
plot.show()
